In [1]:
import numpy as np
import pandas as pd

In [2]:
data2=pd.read_csv("Hotel Price.csv")

In [3]:
data2.head()

,Destination,Condition,Hotel_Rating,Price
0,Bengaluru,Clear,2,4060
1,Udaipur,Clear,5,5840
2,Jodhpur,Clear,1,2470
3,Ooty,Clouds,2,2890
4,Madurai,Rain,3,2420


In [4]:
x=data2.iloc[:,0:3]
y=data2.iloc[:,3]

In [5]:
from sklearn.preprocessing import LabelEncoder
categorical_cols = ["Destination", "Condition"]
numeric_cols = ["Hotel_Rating"]  

# Encode only categorical
label_encoders = {}
for column in categorical_cols:
    le = LabelEncoder()
    x[column] = le.fit_transform(x[column])
    label_encoders[column] = le

# Numeric columns stay as they are
x[numeric_cols] = x[numeric_cols]

In [6]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=0)

In [7]:
from sklearn.preprocessing import StandardScaler
st=StandardScaler()
x_train=st.fit_transform(x_train)
x_test=st.transform(x_test)

In [8]:
from sklearn.ensemble import RandomForestRegressor
rfg=RandomForestRegressor(random_state=0)

In [9]:
rfg.fit(x_train,y_train)

RandomForestRegressor(random_state=0)

In [10]:
import requests

api_key = "1f83e6304f292db0a23225da51a6f712"  
city_name = "Agra"

base_url = "https://api.openweathermap.org/data/2.5/weather?"
complete_url = f"{base_url}q={city_name}&appid={api_key}&units=metric"

response = requests.get(complete_url)
if response.status_code == 200:
    data_api = response.json()
    condition = data_api['weather'][0]['main']  # e.g., "Clear", "Rain"
    temperature = data_api['main']['temp']      # already in Celsius
    wind_speed = data_api['wind']['speed'] * 3.6  # convert m/s → km/h
else:
    raise Exception("API request failed")

In [11]:
print(condition)
print(temperature)
print(wind_speed)

Clear
34.17
10.044


In [12]:
input={
    "Destination":city_name,
    "Condition":condition,
    "Hotel_Rating":5
}

In [13]:
input_df=pd.DataFrame([input])
# Encode categorical only
for column in categorical_cols:
    encoder = label_encoders[column]
    input_df[column] = encoder.transform(input_df[column])

# Keep numeric as-is
input_df[numeric_cols] = input_df[numeric_cols]

# Scale
input_df = st.transform(input_df)

In [14]:
Prediction=rfg.predict(input_df)
print(int(Prediction[0]))

5619


In [15]:
y_pred=rfg.predict(x_test)

In [16]:
y_pred[0:5]

array([2927.41547619, 5666.46946124, 7009.65084249, 2059.37180112,
       2779.81493506])

In [17]:
y_test.head()

398     2990
3833    5300
4836    7060
4572    1990
636     2730
Name: Price, dtype: int64

In [18]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
mean_absolute_error(y_pred,y_test)

125.0488118860025

In [19]:
r2_score(y_pred,y_test)

0.9840775343909455

In [20]:
data2["Price"].describe()

count    5000.000000
mean     3102.942000
std      1360.088531
min      1410.000000
25%      2050.000000
50%      2780.000000
75%      3682.500000
max      9700.000000
Name: Price, dtype: float64

In [21]:
import joblib
joblib.dump(rfg, "price_model.pkl")
joblib.dump(st, "price_scaler.pkl")
joblib.dump(label_encoders, "price_encoders.pkl")
# Removed dumping y_label as it is not defined and not needed

['price_encoders.pkl']